In [8]:
import pandas as pd
import numpy as np
import torch

import time

import sys

import re

from transformers import BertTokenizerFast, BertModel
from datasets import load_dataset

from pprint import pprint
import io

import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

from helper import stream

import psycopg2

# Length of dataset: 2,326,839

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [3]:
data_stream = stream()
next(data_stream).keys()

dict_keys(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed'])

In [4]:
def tokenize_data(data):
    tokenized_data = tokenizer(data, padding=True, truncation=True, return_tensors='pt')
    tokenized_data.to(device)
    return tokenized_data

tokenize_data(next(data_stream)['abstract'])

{'input_ids': tensor([[  101,  2057,  6235,  1037,  2047,  9896,  1010,  1996,  1002,  1006,
          1047,  1010,  1032,  3449,  2140,  1007,  1002,  1011, 21877, 11362,
          2208,  2007,  6087,  1010,  1998,  2224,  2009,  6855,  1037, 23191,
          1997,  1996,  2155,  1997,  1002,  1006,  1047,  1010,  1032,  3449,
          2140,  1007,  1002,  1011, 20288, 19287,  1998,  9896,  2594,  7300,
          2000,  1037,  2155,  1997,  3471,  7175,  3392, 22511,  2015,  1997,
         19287,  1012,  2569, 12107,  1997, 20288, 19287,  3711,  1999, 11841,
          3012,  3399,  1998,  2031,  2363,  3445,  3086,  1999,  3522,  2086,
          1012,  1999,  3327,  1010,  2256,  6910, 28962,  2236,  4697,  1998,
         12919,  1996,  3025,  3463,  1997,  3389,  1998,  2358,  2890,  2378,
          2226,  1998,  2507,  1037,  2047,  6947,  1997,  1996, 10722,  4674,
          1011, 10594,  1011,  3766, 23191,  1997, 19679, 28775,  3723,  1012,
          2057,  2036,  2556,  1037,  

In [5]:
def embed(batch):
    # Tokenize the articles in batches of 10
    tokenized = tokenize_data(batch)  # Assuming tokenize_data accepts a list of articles
    
    with torch.no_grad():
        output = model(**tokenized)
        hidden_states = output.hidden_states
        embedding = hidden_states[-2]
    return embedding

In [10]:
%%time
# Kinda workds? not rlly estimates about a day to embed the whole dataset
def embed_in_batches(data, batch_size=32):
    batch = []
    embeddings = []
    
    for article in data:
        batch.append(article['abstract'])
        
        if len(batch) >= batch_size:
            start_time = time.time()
            embeddings.append(embed(batch))
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f'Batch size: {len(batch)}\tElapsed time: {elapsed_time:0.2f}')
            batch = []
    start_time = time.time()
    embeddings.append(embed(batch))
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Batch size: {len(batch)}\tElapsed time: {elapsed_time:0.2f}')
    return embeddings

CPU times: total: 0 ns
Wall time: 0 ns


In [11]:
%%time
test_emb = embed_in_batches(data_stream)

Batch size: 64	Elapsed time: 0.03


Batch size: 64	Elapsed time: 10.19
Batch size: 64	Elapsed time: 7.73
Batch size: 64	Elapsed time: 7.57


KeyboardInterrupt: 